In [ ]:
from langchain_anthropic import ChatAnthropic
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [4]:

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


In [5]:

llm = ChatAnthropic(model='claude-3-5-sonnet-20240620')

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}



In [6]:


graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()


In [12]:

from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(e)

Install pygraphviz to draw graphs: `pip install pygraphviz`.


In [8]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistent:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("goodbye")
            break
        stream_graph_updates(user_input)
    except:
        pass

goodbye
